In [54]:
#Imports
import featuretools as ft
import pandas as pd

In [55]:
simData = pd.read_csv("./recording_2025_11_27__12_14_17.csv", header=0)
simData = simData.apply(pd.to_numeric, errors="coerce") 

simData["driver_id"] = 0
simData["window_id"] = 0

simData = simData.fillna(0)
simData.isna().sum()
#n = len(simData)
#window_size = n // 4

#simData["window_id"] = (simData.index // window_size).astype(int)

C:\Users\games\AppData\Local\Temp\ipykernel_21048\4097779273.py:1: DtypeWarning: Columns (0,1,6,7,8,13,16,20,24,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,64,65,66,67,68,69,70,71,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  simData = pd.read_csv("./recording_2025_11_27__12_14_17.csv", header=0)


throttle                        0
brakes                          0
rrp_pos                         0
rrp_lin_vel                     0
rrp_quat                        0
                               ..
car0_indicator_right_trigger    0
wheel_factor_adas               0
timestamp                       0
driver_id                       0
window_id                       0
Length: 76, dtype: int64

In [56]:
drivers = simData[["driver_id"]].drop_duplicates().reset_index(drop=True)
drivers = drivers.set_index("driver_id")

windows = simData[["window_id", "driver_id"]].drop_duplicates().reset_index(drop=True)
windows = windows.set_index("window_id")

simData["sample_id"] = range(len(simData))
simData = simData.set_index("sample_id")

es = ft.EntitySet(id="sim_data")

es = es.add_dataframe(
    dataframe_name="drivers",
    dataframe=drivers,
    index="driver_id"
)

es = es.add_dataframe(
    dataframe_name="windows",
    dataframe=windows,
    index="window_id"
)

es = es.add_dataframe(
    dataframe_name="samples",
    dataframe=simData,
    index="sample_id",
    time_index="timestamp"
)

es = es.add_relationship(
    parent_dataframe_name="drivers",
    parent_column_name="driver_id",
    child_dataframe_name="windows",
    child_column_name="driver_id"
)

es = es.add_relationship(
    parent_dataframe_name="windows",
    parent_column_name="window_id",
    child_dataframe_name="samples",
    child_column_name="window_id"
)

c:\Users\games\.conda\envs\MSut311\Lib\site-packages\featuretools\entityset\entityset.py:1733: UserWarning: index driver_id not found in dataframe, creating new integer column
  warnings.warn(
c:\Users\games\.conda\envs\MSut311\Lib\site-packages\featuretools\entityset\entityset.py:1733: UserWarning: index window_id not found in dataframe, creating new integer column
  warnings.warn(
c:\Users\games\.conda\envs\MSut311\Lib\site-packages\featuretools\entityset\entityset.py:1733: UserWarning: index sample_id not found in dataframe, creating new integer column
  warnings.warn(


In [57]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='windows',
    agg_primitives=['sum', 'mean', 'max', 'min', 'count', 'std'],
    max_depth=2,
    verbose=True
)
feature_matrix



Built 754 features
Elapsed: 00:00 | Progress:  39%|███▉      

c:\Users\games\.conda\envs\MSut311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x0000026D559B1620> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
c:\Users\games\.conda\envs\MSut311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x0000026D559B0C20> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  ).agg(to_agg)
c:\Users\games\.conda\envs\MSut311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x0000026D559B00E0> is currently using SeriesGroupBy.sum. In a fut

Elapsed: 00:00 | Progress:  95%|█████████▌

c:\Users\games\.conda\envs\MSut311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x0000026D559B14E0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
c:\Users\games\.conda\envs\MSut311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x0000026D559B00E0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).agg(to_agg)
c:\Users\games\.conda\envs\MSut311\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x0000026D559B0C20> is currently using SeriesGroupBy.min. In a 

Elapsed: 00:01 | Progress: 100%|██████████


,driver_id,COUNT(samples),MAX(samples.brakes),MAX(samples.brakes_vol),MAX(samples.car0_brake_position),MAX(samples.car0_caliper0_quat),MAX(samples.car0_caliper1_quat),MAX(samples.car0_caliper2_quat),MAX(samples.car0_caliper3_quat),MAX(samples.car0_engine_load),...,drivers.SUM(samples.rot_acc),drivers.SUM(samples.rot_vel),drivers.SUM(samples.rot_vel_bc),drivers.SUM(samples.rrp_lin_vel),drivers.SUM(samples.rrp_pos),drivers.SUM(samples.rrp_quat),drivers.SUM(samples.rrp_rot_vel),drivers.SUM(samples.throttle),drivers.SUM(samples.timestamp),drivers.SUM(samples.wheel_factor_adas)
window_id,,,,,,,,,,,,,,,,,,,,,
0,0,39971,0.2925,1.0,0.645105,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4716.15078,2.208131e+07,0.0


In [58]:
feature_defs

[<Feature: driver_id>,
 <Feature: COUNT(samples)>,
 <Feature: MAX(samples.brakes)>,
 <Feature: MAX(samples.brakes_vol)>,
 <Feature: MAX(samples.car0_brake_position)>,
 <Feature: MAX(samples.car0_caliper0_quat)>,
 <Feature: MAX(samples.car0_caliper1_quat)>,
 <Feature: MAX(samples.car0_caliper2_quat)>,
 <Feature: MAX(samples.car0_caliper3_quat)>,
 <Feature: MAX(samples.car0_engine_load)>,
 <Feature: MAX(samples.car0_engine_max_rpm)>,
 <Feature: MAX(samples.car0_engine_rpm)>,
 <Feature: MAX(samples.car0_engine_torque)>,
 <Feature: MAX(samples.car0_gear)>,
 <Feature: MAX(samples.car0_indicator_left_trigger)>,
 <Feature: MAX(samples.car0_indicator_right_trigger)>,
 <Feature: MAX(samples.car0_indicator_trigger)>,
 <Feature: MAX(samples.car0_mask_objects)>,
 <Feature: MAX(samples.car0_mask_objects_adas)>,
 <Feature: MAX(samples.car0_rpm)>,
 <Feature: MAX(samples.car0_steer_quat)>,
 <Feature: MAX(samples.car0_throttle_position)>,
 <Feature: MAX(samples.car0_turbo_pressure)>,
 <Feature: MAX(sam